In [1]:
from import_zig import import_zig

# Types of sources

There are three ways to import Zig code. Source string, source file and source directory.

In [2]:
module = import_zig(source_code="pub fn one() i32 {return 1;}")
module.one()

1

In [3]:
module = import_zig(file="single_file.zig")
module.add(40, 2)

42

In [4]:
module = import_zig(directory="multiple_files")
module.add_constant(10)

15

# Compiling to a binary

It's also possible to output a binary, which can then be imported again, if it is on Python's path. The below example places the binary in the current working directory, so that it can be imported directly.

The binary is platform specific.

In [5]:
from import_zig import compile_to
compile_to(target_dir=".", module_name="my_module", file="single_file.zig")

In [6]:
import my_module
my_module.add(2, 2)

4

# Type mapping

The following shows how types are mapped between Zig and Python. Types can be nested arbitrarely

In [7]:
module = import_zig(source_code=r"""
const std = @import("std");

const allTypes = struct {
    i: i64,
    f: f64,
    b: bool,
    a: [3]i42,
    l: []const i64, // slice
    s: []const u8, // string (utf8)
    st: struct { a: u8, b: i64 }, // anonymous struct, resulting Python type will still be named
    t: std.meta.Tuple(&[_]type{ u8, i64 }), // tuple, will turn into a Python tuple
    o: ?i64, // optionals are possible
};

pub fn testTypes(param: allTypes) allTypes {
    std.debug.print("From Zig: {any}\n", .{param});
    return param;
}
""")

print("From Python:", module.testTypes([
    5, 1.5, "truthy", [1, 2, 3], range(5), "hey", (2, 3), (2, 3), None
]))

From Python: import_zig.allTypes(i=5, f=1.5, b=True, a=[1, 2, 3], l=[0, 1, 2, 3, 4], s='hey', st=import_zig.allTypes__struct_8074(a=2, b=3), t=(2, 3), o=None)


From Zig: inner.import_fns.allTypes{ .i = 5, .f = 1.5e0, .b = true, .a = { 1, 2, 3 }, .l = { 0, 1, 2, 3, 4 }, .s = { 104, 101, 121 }, .st = inner.import_fns.allTypes.allTypes__struct_8074{ .a = 2, .b = 3 }, .t = { 2, 3 }, .o = null }


# Python C API

It's possible to interact with the C API directly

In [8]:
module = import_zig(source_code=r"""
const pyu = @import("../py_utils.zig");
const py = pyu.py;

pub fn indexList(py_list: *py.PyObject, index: isize) *py.PyObject {
    const borrowed = py.PyList_GetItem(py_list, index) orelse unreachable;
    return py.Py_NewRef(borrowed);
}
""")
module.indexList([1,2,3], 1)

2

# Errors

Python often indicates errors with null pointers. In the above example we deal with a potential indexing error with the `orelse unreachable` code block, which tells Zig to crash the program if `PyList_GetItem` is unsuccessful.

Instead we might want to propagate this exception back to the Python interpreter.

In [10]:
module = import_zig(source_code=r"""
const pyu = @import("../py_utils.zig");
const py = pyu.py;

pub fn indexList(py_list: *py.PyObject, index: isize) !*py.PyObject {
    const borrowed = py.PyList_GetItem(py_list, index) orelse return pyu.PyErr.PyErr;
    return py.Py_NewRef(borrowed);
}
""")
module.indexList([1,2,3], 5)

IndexError: list index out of range

We can also produce our own exceptions, either by setting an exception using the provided utility function or by returning some other Zig error, which will get wrapped into an exception

In [12]:
module = import_zig(source_code=r"""
const pyu = @import("../py_utils.zig");
const py = pyu.py;

pub fn pyExc(value: i64) !*py.PyObject {
    return pyu.raise(.Exception, "The value was {}", .{value});
}

const err = error{myerr};

pub fn zigErr() !*py.PyObject {
    return err.myerr;
}
""")
module.pyExc(5)

Exception: The value was 5

In [13]:
module.zigErr()

Exception: Zig function returned an error: error.myerr